In [21]:
# -*- coding: utf-8 -*-

import numpy as np 
import pandas as pd 
import keras 
import jieba
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import Dense,Dropout,Activation,Input,Convolution1D,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Flatten,concatenate,Embedding,GRU,Lambda, LSTM, TimeDistributed
from keras.layers.merge import Concatenate
from keras.models import model_from_json
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.layers import Dense, Embedding, LSTM,Flatten
from gensim.models import word2vec
import time
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
np.set_printoptions(threshold=100)

In [2]:
def find_str(x):
    x1 = x.split('->')[:4]
    return '->'.join(x1)

rawdata_path = './raw_data/4月份数据.csv'
stopkeyname = './raw_data/中文停用词表.txt'
user_dict = './raw_data/userdict.txt'
#读取文件
df = pd.read_csv(rawdata_path)
jieba.load_userdict(user_dict)
stopkey ={}.fromkeys([line.rstrip() for line in open(stopkeyname,encoding='utf-8')])  
#生成新的业务类别
df['business_type_new'] = df['business_type'].apply(lambda x : find_str(x))
type_count = pd.DataFrame({'COUNT':df.groupby(['business_type_new']).size()}).reset_index()
filter_less = type_count[type_count.COUNT>50].business_type_new.tolist()
df = df[df.business_type_new.isin(filter_less)].reset_index()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.713 seconds.
Prefix dict has been built succesfully.


In [3]:
import re
def isNum(value):
    try:
        value + 1
    except TypeError:
        return False
    else:
        return True
    

def isNum(value):
    try:
        value + 1
    except TypeError:
        return False
    else:
        return True

def cleaner(word):
    word = re.sub(r'\#\.', '', word)
    word = re.sub(r'\n', '', word)
    word = re.sub(r',', '', word)
    word = re.sub(r'\-', ' ', word)
    word = re.sub(r'\.', '', word)
    word = re.sub(r'\\', ' ', word)
    word = re.sub(r'\\x\.+', '', word)
    word = re.sub(r'\d', '', word)
    word = re.sub(r'^_.', '', word)
    word = re.sub(r'_', ' ', word)
    word = re.sub(r'^ ', '', word)
    word = re.sub(r' $', '', word)
    word = re.sub(r'\?', '', word)
    word = re.sub(r'|', '', word)
    word = re.sub(r'（', '', word)
    word = re.sub(r'）', '', word)
    word = re.sub(r':', '', word)
    word = re.sub(r'null', '', word)
    word = re.sub(r'Null', '', word)
    word = re.sub(r'[\[,\] ,:,),|,(,。,，,/, ：,【,】,;,！,、,？,]', '', word)
    return word

x_list_1 = []
for i in range(len(df['CONTENT_'])):
    word_1=[]
    result_1 = jieba.cut(str(df['CONTENT_'].iloc[i]))
    for segs in result_1:
        segs = cleaner(segs)
        if segs != ''and isNum(segs) != True:
            word_1.append(segs)
    x_list_1.append(' '.join(word_1))
df['content_new'] = x_list_1
#进行tfidf的维表数据

#生成维表数据
'''
tf_train = TfidfVectorizer()
f1 = open('./raw_data/vocabulary_op.json','wb')
file = tf_train.vocabulary_
pickle.dump(file,f1)
f1.close()
idfs = tf_train.idf_
np.save('./raw_data/idfs_op.npy',idfs)
'''

# for label in df['business_type_new'].unique():
#     while df.loc[df['business_type_new']==label].shape[0]<100:
#         df = df.append(df.loc[df['business_type_new']==label],ignore_index=True)
#训练tfidf


"\ntf_train = TfidfVectorizer()\nf1 = open('./raw_data/vocabulary_op.json','wb')\nfile = tf_train.vocabulary_\npickle.dump(file,f1)\nf1.close()\nidfs = tf_train.idf_\nnp.save('./raw_data/idfs_op.npy',idfs)\n"

In [4]:
from keras.preprocessing.text import Tokenizer
max_word_len = 4000
tokenizer = Tokenizer(
    nb_words=max_word_len, 
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   split=' ')
tokenizer.fit_on_texts(df['content_new'])

X=tokenizer.texts_to_sequences(df['content_new'])
X = pad_sequences(X)

d:\programtools\anaconda\envs\tensorflow\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [5]:
X.shape

(34559, 213)

In [6]:
train_x,test_x,train_y,test_y = train_test_split( X,df['business_type_new'],test_size = 0.1, random_state=2019) 

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder().fit(train_y)
train_y = le.transform(train_y)
test_y = le.transform(test_y)
train_y_cat = to_categorical(train_y,num_classes=39)
test_y_cat = to_categorical(test_y,num_classes=39)

In [15]:
def word2vec_train():

	sentences = word2vec.Text8Corpus("cuttext.txt")
	model = word2vec.Word2Vec(sentences)
	model.save('./word2vec.m')
#	print(model.similarity('被害人','发生'))
	# print(model.most_similar("被告人"))
	print('finished and saved!')
	return model

In [9]:
def word2vec_train():

	sentences = word2vec.Text8Corpus("cuttext.txt")
	model = word2vec.Word2Vec(sentences)
	model.save('./word2vec.m')

	print('finished and saved!')
	return model


In [22]:
np.savetxt("cuttext.txt",df['content_new'].values,fmt="%s")
w2v_model = word2vec_train()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc4 in position 0: invalid continuation byte

In [10]:
def baseline_model(y,max_features,embedding_dims,filters):
	kernel_size = 3

	model = Sequential()
	model.add(Embedding(max_features, embedding_dims))        # 使用Embedding层将每个词编码转换为词向量
	model.add(Conv1D(filters,
					 kernel_size,
					 padding='valid',
					 activation='relu',
					 strides=1))
	# 池化
	model.add(GlobalMaxPooling1D())

	model.add(Dense(y.shape[1], activation='softmax')) #第一个参数units: 全连接层输出的维度，即下一层神经元的个数。
	model.add(Dropout(0.2))
	model.compile(loss='categorical_crossentropy',
				  optimizer='adam',
				  metrics=['accuracy'])

	model.summary()

	return model


In [11]:
model = baseline_model(train_y_cat,max_word_len,128,200)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         512000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 200)         77000     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 39)                7839      
_________________________________________________________________
dropout_1 (Dropout)          (None, 39)                0         
Total params: 596,839
Trainable params: 596,839
Non-trainable params: 0
_______________

In [12]:
 earlyStopping = keras.callbacks.EarlyStopping(
			monitor='val_loss',
			patience=1,  # 当early stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练。
			verbose=0,
			mode='auto')

print("training model")
model.fit(train_x,train_y_cat,validation_split=0.2,batch_size=32,epochs=10,verbose=2,callbacks=[earlyStopping],shuffle=True,class_weight='auto') #自动设置class weight让每类的sample对损失的贡献相等


training model
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 24882 samples, validate on 6221 samples
Epoch 1/10
 - 10s - loss: 3.9804 - acc: 0.5788 - val_loss: 0.7427 - val_acc: 0.7542
Epoch 2/10
 - 7s - loss: 3.5873 - acc: 0.6557 - val_loss: 0.6332 - val_acc: 0.7885
Epoch 3/10
 - 7s - loss: 3.5510 - acc: 0.6797 - val_loss: 0.6274 - val_acc: 0.7860
Epoch 4/10
 - 7s - loss: 3.4858 - acc: 0.7054 - val_loss: 0.5833 - val_acc: 0.8060
Epoch 5/10
 - 7s - loss: 3.4738 - acc: 0.7223 - val_loss: 0.6129 - val_acc: 0.8061


In [13]:
loss, accuracy = model.evaluate(test_x, test_y_cat)
print('test loss: ', loss)
print('test accuracy: ', accuracy)
#np.savetxt('res_{}_{}.csv'.format(time.strftime("%Y%m%d"),accuracy),res_index)
model.save('text_cnn_{}_{}.h5'.format(time.strftime("%Y%m%d"),accuracy))

3456/3456 [==============================] - 0s 74us/step
test loss:  0.5831887171500258
test accuracy:  0.8041087962962963


In [ ]:
def test_cnn(y,maxlen,max_features,embedding_dims,filters = 250):
	#Inputs
	seq = Input(shape=[maxlen],name='x_seq')

	#Embedding layers
	emb = Embedding(max_features,embedding_dims)(seq)

	# conv layers
	convs = []
	filter_sizes = [2,3,4,5]
	for fsz in filter_sizes:
		conv1 = Conv1D(filters,kernel_size=fsz,activation='tanh')(emb)
		pool1 = MaxPooling1D(maxlen-fsz+1)(conv1)
		pool1 = Flatten()(pool1)
		convs.append(pool1)
	merge = concatenate(convs,axis=1)

	out = Dropout(0.5)(merge)
	output = Dense(32,activation='relu')(out)

	output = Dense(units=y.shape[1],activation='sigmoid')(output)

	model = Model([seq],output)
	model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
	return model


In [ ]:
def cnn_w2v(y,max_features,embedding_dims,filters,maxlen):
	# CNN参数
	kernel_size = 3

	model = gensim.models.Word2Vec.load('./predictor/model/word2vec')
	word2idx = {"_PAD": 0}  # 初始化 `[word : token]` 字典，后期 tokenize 语料库就是用该词典。
	vocab_list = [(k, model.wv[k]) for k, v in model.wv.vocab.items()]
	# 存储所有 word2vec 中所有向量的数组，留意其中多一位，词向量全为 0， 用于 padding
	embeddings_matrix = np.zeros((len(model.wv.vocab.items()) + 1, model.vector_size))
	print('Found %s word vectors.' % len(model.wv.vocab.items()))
	for i in range(len(vocab_list)):
		word = vocab_list[i][0]
		word2idx[word] = i + 1
		embeddings_matrix[i + 1] = vocab_list[i][1]

	model = Sequential()
	# 使用Embedding层将每个词编码转换为词向量
	model.add(Embedding(len(embeddings_matrix),       #表示文本数据中词汇的取值可能数,从语料库之中保留多少个单词。 因为Keras需要预留一个全零层， 所以+1
								embedding_dims,       # 嵌入单词的向量空间的大小。它为每个单词定义了这个层的输出向量的大小
								weights=[embeddings_matrix], #构建一个[num_words, EMBEDDING_DIM]的矩阵,然后遍历word_index，将word在W2V模型之中对应vector复制过来。换个方式说：embedding_matrix 是原始W2V的子集，排列顺序按照Tokenizer在fit之后的词顺序。作为权重喂给Embedding Layer
								input_length=maxlen,     # 输入序列的长度，也就是一次输入带有的词汇个数
								trainable=False        # 我们设置 trainable = False，代表词向量不作为参数进行更新
						))
	model.add(Conv1D(filters,
					 kernel_size,
					 padding='valid',
					 activation='relu',
					 strides=1))
	# 池化
	model.add(GlobalMaxPooling1D())

	model.add(Dense(y.shape[1], activation='softmax')) #第一个参数units: 全连接层输出的维度，即下一层神经元的个数。
	model.add(Dropout(0.2))
	model.compile(loss='categorical_crossentropy',
				  optimizer='adam',
				  metrics=['accuracy'])

	model.summary()

	return model
